In [1]:
import pandas as pd
import plotly as pt
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
confirmed_cases = pd.read_csv('Data/time_series_covid19_confirmed_global.csv')

country_coordinates = {}

for index,row in confirmed_cases.iterrows():
    if row['Country/Region'] in country_coordinates:
        pass
    else:
        country_coordinates[row['Country/Region']] = (row.Lat, row.Long)

In [3]:
confirmed_cases_by_country = confirmed_cases.groupby("Country/Region").sum(numeric_only=True)\
                                            .drop(columns=['Lat', 'Long'])

In [4]:
def date_converter(date_str):
    month_map = {'1': "January", '2': "February", '3': "March", '4': "April", '5': "May", '6': "June", 
                 '7': "July", '8': "August", '9': "September", '10': "October", '11': "November", '12': "December"}

    split_date = date_str.split('/')

    return month_map[split_date[0]] + ' ' + '20' + split_date[-1]

In [5]:
month_list = []
for i in confirmed_cases_by_country.columns:
    month_list.append(date_converter(i))

confirmed_cases_by_country.columns = month_list

In [6]:
monthly_data_df = pd.DataFrame()

for month in pd.unique(confirmed_cases_by_country.columns):
    monthly_data_df[month] = confirmed_cases_by_country[month].max(axis=1)

monthly_data_df

,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,October 2020,...,June 2022,July 2022,August 2022,September 2022,October 2022,November 2022,December 2022,January 2023,February 2023,March 2023
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,5,166,1827,15180,31445,36628,38248,39354,41334,...,182528,185749,193250,199188,203063,205907,207559,208545,209322,209451
Albania,0,0,243,773,1137,2535,5276,9513,13649,20875,...,280851,312097,329352,332221,332966,333343,333806,334167,334391,334457
Algeria,0,1,716,4006,9394,13907,30394,44494,51530,57942,...,266087,267454,270304,270673,270838,271090,271228,271378,271441,271496
Andorra,0,0,376,745,764,855,925,1176,2050,4756,...,43774,45508,46027,46227,46535,47219,47751,47839,47866,47890
Angola,0,0,7,27,86,284,1148,2654,4972,10805,...,101320,102301,102636,103131,103131,104676,105095,105184,105255,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,119,344,448,2428,11837,22729,39899,53520,...,659853,682096,702332,702768,703036,703036,703228,703228,703228,703228
Winter Olympics 2022,0,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
Yemen,0,0,0,6,323,1158,1728,1958,2034,2063,...,11824,11877,11926,11935,11939,11945,11945,11945,11945,11945


In [8]:
monthly_data_df.to_csv('cleaned_data')

In [ ]:
globe = folium.Map(location=[54.5, 15.25], zoom_start=2)

final_df = monthly_data_df.max(axis=1).to_frame(name='max_cases')
max_val = final_df['max_cases'].max()
min_val = final_df['max_cases'].min()

final_df['scaled_confirmed_cases'] = 1 + ((final_df['max_cases'] - min_val) * 9) /(max_val-min_val)

for country in country_coordinates:
    country_name = country
    ranked_score = final_df.loc[country_name, 'max_cases']

    html = f"""
    <h4 style="margin-bottom: 5px;">Country:</h4>
    <p><strong>{country_name}</strong></p>

    <h4 style="margin-bottom: 5px;">Total Cumulative Cases:</h4>
    <p><strong>{ranked_score:.2f}</strong></p>
    """
    
    folium.CircleMarker(
        location=country_coordinates[country],
        radius=final_df.loc[country, 'scaled_confirmed_cases'],
        fill=True,
        popup=folium.Popup(html, max_width=300)
    ).add_to(globe)

display(globe)